# RCFM Model

## 1. Import modules and helper functions

In [1]:
# Modules
import matplotlib.pyplot as plt
import numpy as np
from math import sqrt
from scipy.optimize import curve_fit

# Helper functions from DataAid.py and DataImport.py
import DataAid
import DataImporter

# Numerically stable class of functions from Neros_v2.py
import Neros_v4

## 2. Load Galaxy Data

In [2]:
# Load Galaxy Data
sparcGalaxies = DataAid.GetGalaxyData("data/Sparc/Rotmod_LTG/")
littleDataGalaxies = DataAid.GetGalaxyData("data/little-data-things/data/")
lcmGalaxies = DataAid.GetGalaxyData("data/LCMFits/data/")

# Load Milky Way Model Data
xueSofueGalaxies = DataAid.GetGalaxyData("data/XueSofue/")
mcGaughMW = DataAid.GetGalaxyData("data/McGaugh/")

# Create array of Milky Way radius and vlum tuples from model data
MWXueSofue = np.array(xueSofueGalaxies['MW_lum'])
MWMcGaugh = np.array(mcGaughMW['MW_lumMcGaugh'])


## 3. Create Neros class instance

In [3]:
# Create Neros instance to perform calculations with the supplied Milky Way model as comparison
# Change Milky Way model by changing the variable in the parentheses
# i.e. neros_fns = Neros_v2.Neros(MWModelVariable)

neros_fns = Neros_v4.Neros(MWXueSofue)
MW_name = "XueSofue" # Change this if you change the MW model in neros_fns!
MW_rad = neros_fns.mw_rad
MW_vLum = neros_fns.mw_vLum
#MW_phi = neros_fns.mw_phi

MW_vLum_interp_func = neros_fns.mw_vLum_interp

## 4. Designate outfile name, initialize variables, and designate galaxy sample

In [4]:
# This designates which galaxy sample to fit
galaxies = lcmGalaxies


## 5. Fit galaxies, print and save graphs

In [ ]:
"""
This is the main body of the model. 
It loops through galaxies in the designated sample,
it calculates vLCM, vNeros, and chiSquared,
it fits galaxy data using alpha and vLumFreeParam,
it prints the values of alpha and vLumFreeParam to the console,
and it saves the graphs to a file.
"""

# This is where the fitted alpha & vlum free parameter will be written for each galaxy
out_file = "imported-data/data_" + str(MW_name) + ".csv"
with open(out_file, 'w') as f:
    f.write('{0},{1},{2},{3},{4},{5}\n'.format("Galaxy", "chi_square",  "alpha", "disk_scale", "bulge_scale","phi_zero"))
f.close()

total_chi_squared = []


for galaxyName in galaxies:
    f, ax = plt.subplots(1, figsize = (15, 15))
    
    # Extract out the needed galaxy components
    galaxy = np.array(galaxies[galaxyName])
    galaxy_rad = galaxy[:,0]
    galaxy_vObs = galaxy[:,1]
    galaxy_error = galaxy[:,2]
    galaxy_gas = galaxy[:,3]
    galaxy_disk = galaxy[:,4]
    galaxy_bulge = galaxy[:,5]

    # Just fit then extract the relevant pieces
    neros_fns.fit(galaxy_rad, galaxy_gas, galaxy_disk, galaxy_bulge, galaxy_vObs, galaxy_error)

    print(f"galaxyName is: {galaxyName}")
    fit_results = neros_fns.get_fit_results(galaxy_rad)
    for param in fit_results:
        print(f"{param} is: {fit_results[param]}")
    print("--------------------------")
    
    total_chi_squared.append(fit_results['chi_squared'])
    
    print("Running average chi_squared: {}".format(sum(total_chi_squared)/len(total_chi_squared)))

    
    # Write galaxyName, fitted parameters
    with open(out_file, 'a') as f:
        f.write(f"{galaxyName},{fit_results['chi_squared']},{fit_results['alpha']}, \
                {fit_results['disk_scale']},{fit_results['bulge_scale']},{fit_results['phi_zero']} \n")
    f.close()

    trimmed_rad = neros_fns.get_rad()
    trimmed_vLum_updated = neros_fns.get_vLum_scaled()
    trimmed_vObs = neros_fns.get_vObs()
    trimmed_error = neros_fns.get_vObsError()
    vNeros = neros_fns.get_vNeros()
    
    # plot
    # y-axis scales to the maximum velocity value in the list galaxy_vObs_err_incl
    # or to the maximum value in the list vNeros, whichever is the bigger number
    y_max = max(max(trimmed_vObs + trimmed_error), max(vNeros))
    ax.set_ylim(bottom = 0, top = y_max + 15)

    # plot vObs and vNeros and updated vLum
    ax.plot(trimmed_rad, vNeros, label="{}_vNeros".format(galaxyName))
    ax.plot(trimmed_rad, trimmed_vObs, label="{}_vObs".format(galaxyName))
    ax.plot(trimmed_rad, trimmed_vLum_updated, label="{}_new_vLum".format(galaxyName))
    ax.plot([],[], ' ', label="$\chi^2$ = {}".format(fit_results['chi_squared']))

    # error bar in vObs
    for i in range(len(trimmed_rad)):
        ax.vlines(trimmed_rad[i], trimmed_vObs[i] - trimmed_error[i], trimmed_vObs[i] + trimmed_error[i])

    ax.legend(loc="upper right")

    graph_file_name = "graphs/" + str(galaxyName) + "_" + str(MW_name)

    plt.savefig(graph_file_name)


/Users/sophiacisneros/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


galaxyName is: NGC3198_deBlok
chi_squared is: 1.395070438213474
alpha is: 188.0031904481162
disk_scale is: 1.206889554409153
bulge_scale is: 1.0
phi_zero is: 3.694658858819989e-06
--------------------------
Running average chi_squared: 1.395070438213474
galaxyName is: NGC2841_deBlokTHINGS
chi_squared is: 1.3063860500096636
alpha is: 22.049366053691376
disk_scale is: 1.3402202753646881
bulge_scale is: 1.1806625623166718
phi_zero is: 4.300785293848209e-06
--------------------------
Running average chi_squared: 1.3507282441115689
galaxyName is: M31_Carignan_SNC
chi_squared is: 0.30429071077322484
alpha is: 23.0565428828463
disk_scale is: 1.0641779884613194
bulge_scale is: 0.9449875546283723
phi_zero is: 3.729889199668195e-06
--------------------------
Running average chi_squared: 1.0019157329987876


/Users/sophiacisneros/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


galaxyName is: NGC4088_rotmod-Copy1
chi_squared is: 0.38621046243136076
alpha is: 149.42109451778222
disk_scale is: 0.6585167791154707
bulge_scale is: 1.0
phi_zero is: 3.1919951620190414e-06
--------------------------
Running average chi_squared: 0.8479894153569308


/Users/sophiacisneros/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


...

galaxyName is: NGC2403_deBlokThings
chi_squared is: 1.2867002080689685
alpha is: 372.43784774875337
disk_scale is: 1.3445572186931087
bulge_scale is: 1.0
phi_zero is: 4.432909492450046e-06
--------------------------
Running average chi_squared: 1.367373206693796


/Users/sophiacisneros/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


galaxyName is: NGC3198_GentileMaria
chi_squared is: 1.927568169160115
alpha is: 135.21599221046336
disk_scale is: 1.599089599436129
bulge_scale is: 1.0
phi_zero is: 3.664487332755443e-06
--------------------------
Running average chi_squared: 1.4296170914122759
galaxyName is: M31_Carignan_SNC2
chi_squared is: 0.27543109252584796
alpha is: 25.422622939233726
disk_scale is: 1.0424938337347878
bulge_scale is: 0.9676608323214725
phi_zero is: 3.729889199668195e-06
--------------------------
Running average chi_squared: 1.314198491523633


/Users/sophiacisneros/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


galaxyName is: F563_1McGaugh
chi_squared is: 0.13167845981242854
alpha is: 267.1097879044988
disk_scale is: 1.6075307815430255
bulge_scale is: 1.0
phi_zero is: 3.392034252601659e-06
--------------------------
Running average chi_squared: 1.206696670458978
